# Загрузка Pandas и очистка данных

In [38]:
import pandas as pd
import numpy as np
import ast
import datetime
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics

import sqlite3

In [39]:
df = pd.read_csv('main_task.xls')

print('read')
print(df.info())

read
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


In [40]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [41]:
'''
Changing price values from $, $$-$$$, $$$$ into 1, 3 and 5 accordingly.
Pulling (with get_price.py) some missing price values from web to check if it makes any difference
'''
def convert_price(string):
    if string == '$$$$':
        return 5
    elif string == '$':
        return 1
    else:
        return 3

con = sqlite3.connect('main_task.db')
cur = con.cursor()
cur.execute(f"SELECT * from prices")
updated_prices = df['Price Range'].to_dict()
updated_prices.update({idx: value for idx, value in cur.fetchall()})
df['Price_Range_upd'] = pd.Series(updated_prices)

df['Avg_Price'] = df['Price Range'].apply(convert_price)
df['Avg_Price_upd'] = df['Price_Range_upd'].apply(convert_price)

print('price')
print(df.info())

price
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
 10  Price_Range_upd    40000 non-null  object 
 11  Avg_Price          40000 non-null  int64  
 12  Avg_Price_upd      40000 non-null  int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 4.0+ MB
None


In [42]:
'''
0. processing cuisines to first build a shorter list (down from 125)
1. checking which restaurants are serving national cuisines (based on the city)
2. counting number of cuisines served by each restaurant
3. building dummy list from the short cuisine list
4. creating new property of predicted quality based on ratio of served cuisines to price
'''

def build_top_list(series, top):
    cuisines = []
    for string in series:
        if type(string) == str:
            cuisines.extend(ast.literal_eval(string))
    return [x[0] for x in Counter(cuisines).most_common(top)]

def check_national_cuisines(series):
    city_national_cuisine = {'Paris': 'French',
                             'Stockholm': 'Swedish',
                             'London': 'British',
                             'Berlin': 'German',
                             'Munich': 'German',
                             'Oporto': 'Portuguese',
                             'Milan': 'Italian',
                             'Vienna': 'Austrian',
                             'Rome': 'Italian',
                             'Barcelona': 'Spanish',
                             'Madrid': 'Spanish',
                             'Dublin': 'Irish',
                             'Brussels': 'Belgian',
                             'Zurich': 'Swiss',
                             'Warsaw': 'Polish',
                             'Budapest': 'Hungarian',
                             'Copenhagen': 'Danish',
                             'Amsterdam': 'Dutch',
                             'Lyon': 'French',
                             'Hamburg': 'German',
                             'Lisbon': 'Portuguese',
                             'Prague': 'Czech',
                             'Oslo': 'Norwegian',
                             'Edinburgh': 'Scottish',
                             'Geneva': 'Swiss',
                             'Ljubljana': 'Slovenian',
                             'Athens': 'Greek',
                             'Luxembourg': 'Central European',
                             'Krakow': 'Polish',
                             }
    result = {}
    for idx, cuisines in series.iteritems():
        if type(cuisines) == str:
            city = df['City'][idx]
            if city_national_cuisine.get(city) in ast.literal_eval(cuisines):
                result[idx] = 1
            else:
                result[idx] = 0
        else:
            result[idx] = 0
    return pd.Series(result)

def regroup_cuisines(cuisines, top_list):
    if type(cuisines) == str:
        result = set()
        for cuisine in ast.literal_eval(cuisines):
        # regrouping some common cuisine categories
            if cuisine in ['Bar', 'Pub', 'Wine Bar']:
                result.add('Bar')
            elif cuisine in ['Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options', 'Healthy']:
                result.add('Healthy')
            elif cuisine in ['Asian', 'Japanese', 'Sushi', 'Chinese', 'Thai']:
                result.add('Asian')
            elif cuisine in ['Fast Food', 'American']:
                result.add('Fast Food')
            # processing other most common cuisines
            elif cuisine in top_list:
                result.add(cuisine)
            # replacing all other cuisines with 'Others' category
            else:
                result.add('Others')
        return list(result)

top_list = build_top_list(df['Cuisine Style'], top=30)
df['Cuisine_National'] = check_national_cuisines(df['Cuisine Style'])
df['Cuisine_Styles_Short'] = df['Cuisine Style'].apply(lambda x: regroup_cuisines(x, top_list))
df['Cuisine_Style_Count'] = df['Cuisine_Styles_Short']\
    .apply(lambda x: len(x) if type(x) == list else 3)  #using 3 instead of 1 as it gives better results

df_dummies = pd.get_dummies(df['Cuisine_Styles_Short'].apply(pd.Series).stack()).sum(level=0)
df_dummies = df_dummies.reindex(df.index, fill_value=0)

df = pd.merge(left=df, right=df_dummies, how='left', left_index=True, right_index=True)

df['Cuisine_Predicted_Quality'] = df['Avg_Price_upd'] / df['Cuisine_Style_Count']

print('cuisines')
print(df.info())

cuisines
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Restaurant_id              40000 non-null  object 
 1   City                       40000 non-null  object 
 2   Cuisine Style              30717 non-null  object 
 3   Ranking                    40000 non-null  float64
 4   Rating                     40000 non-null  float64
 5   Price Range                26114 non-null  object 
 6   Number of Reviews          37457 non-null  float64
 7   Reviews                    40000 non-null  object 
 8   URL_TA                     40000 non-null  object 
 9   ID_TA                      40000 non-null  object 
 10  Price_Range_upd            40000 non-null  object 
 11  Avg_Price                  40000 non-null  int64  
 12  Avg_Price_upd              40000 non-null  int64  
 13  Cuisine_National           40000 non-

In [43]:
'''
0. calculating days since last review (considering the 2018-02-26 is the last date) and days between reviews
1. replacing outliers with median (for days b/w reviews) and max (for days since last review)
2. replacing outliers with max value (excluding outlier values) amount of reviews
3. adding data on max rank per city
'''
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return {'min': lower_bound, 'max': upper_bound}

def remove_outliers(value, outliers):
    if outliers.get('max') > value > outliers.get('min'):
        return value
    else:
        return

def days_since_review(string):
    max_date = datetime.datetime.strptime('2018-02-26', '%Y-%m-%d')
    if type(string) == str:
        try:
            content = ast.literal_eval(string)
            if len(content)>0:
                l_dates = [datetime.datetime.strptime(x, '%m/%d/%Y') for x in content[1]]
                if len(l_dates) > 0:
                    min_date = max(l_dates)
                return (max_date - min_date).days
        except:
            return

def days_between_reviews(string):
    if type(string) == str:
        try:
            content = ast.literal_eval(string)
            if len(content)>0:
                l_dates = [datetime.datetime.strptime(x, '%m/%d/%Y') for x in content[1]]
                if len(l_dates) >= 2:
                    min_date = min(l_dates)
                    max_date = max(l_dates)
                    return (max_date - min_date).days
        except:
            return

df['Days_Since_Review'] = df['Reviews'].apply(days_since_review)
outliners = outliers_iqr(df['Days_Since_Review'].dropna())
df['Days_Since_Review'] = df['Days_Since_Review'].apply(lambda x: remove_outliers(x, outliners))
df['Days_Since_Review'].fillna(value=df['Days_Since_Review'].max(), inplace=True)

df['Days_Between_Reviews'] = df['Reviews'].apply(days_between_reviews)
outliners = outliers_iqr(df['Days_Between_Reviews'].dropna())
df['Days_Between_Reviews'] = df['Days_Between_Reviews'].apply(lambda x: remove_outliers(x, outliners))
df['Days_Between_Reviews'].fillna(value=df['Days_Between_Reviews'].median(), inplace=True)

outliners = outliers_iqr(df['Number of Reviews'].dropna())
df['Number_of_Reviews'] = df['Number of Reviews'].apply(lambda x: remove_outliers(x, outliners))
df['Number_of_Reviews'].fillna(value=df['Number_of_Reviews'].max(), inplace=True)

city_max_rank = {}
for city in df['City'].unique():
    city_max_rank[city] = df.query('City == @city')['Ranking'].max()
df['City_max_Rank'] = df['City'].replace(city_max_rank)

print('rates')
print(df.info())

rates
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Restaurant_id              40000 non-null  object 
 1   City                       40000 non-null  object 
 2   Cuisine Style              30717 non-null  object 
 3   Ranking                    40000 non-null  float64
 4   Rating                     40000 non-null  float64
 5   Price Range                26114 non-null  object 
 6   Number of Reviews          37457 non-null  float64
 7   Reviews                    40000 non-null  object 
 8   URL_TA                     40000 non-null  object 
 9   ID_TA                      40000 non-null  object 
 10  Price_Range_upd            40000 non-null  object 
 11  Avg_Price                  40000 non-null  int64  
 12  Avg_Price_upd              40000 non-null  int64  
 13  Cuisine_National           40000 non-nul

In [44]:
'''
dropping some columns before processing the df
'''
df_result = df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Price_Range_upd', 'Reviews', 'URL_TA',
                     'ID_TA', 'Cuisine_Styles_Short', 'Number of Reviews', 'Avg_Price',
                     #'City_max_Rank',
                     #'Ranking',
                     #'Cuisine_Predicted_Quality',
                     #'Avg_Price_upd',
                     #'Cuisine_National',
                     #'Cuisine_Style_Count',
                     #'Asian', 'Bar', 'British', 'Cafe', 'Central European', 'European', 'Fast Food', 'French', 'German',
                     #'Greek', 'Healthy', 'Indian', 'International', 'Italian', 'Mediterranean', 'Middle Eastern',
                     #'Others', 'Pizza', 'Portuguese', 'Seafood', 'Spanish',
                     #'Days_Since_Review',
                     #'Days_Between_Reviews',
                     #'Number_of_Reviews',
                     ], axis=1)

print(df_result.columns)

X = df_result.drop(['Rating'], axis = 1)
y = df_result['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

Index(['Ranking', 'Rating', 'Avg_Price_upd', 'Cuisine_National',
       'Cuisine_Style_Count', 'Asian', 'Bar', 'British', 'Cafe',
       'Central European', 'European', 'Fast Food', 'French', 'German',
       'Greek', 'Healthy', 'Indian', 'International', 'Italian',
       'Mediterranean', 'Middle Eastern', 'Others', 'Pizza', 'Portuguese',
       'Seafood', 'Spanish', 'Cuisine_Predicted_Quality', 'Days_Since_Review',
       'Days_Between_Reviews', 'Number_of_Reviews', 'City_max_Rank'],
      dtype='object')
MAE: 0.21174400000000002
